In [183]:
import graphviz as gv

In [184]:
'''The next thing on the todo list is to make different colors/styles for extinct or extant species.
Maybe add graph title to function, so the label could be "Permian" or "Mammals" or something else'''

'The next thing on the todo list is to make different colors/styles for extinct or extant species.\nMaybe add graph title to function, so the label could be "Permian" or "Mammals" or something else'

In [185]:
#We'll use functools to make things a little quicker.
import functools
graph = functools.partial(gv.Graph, format='svg')
digraph = functools.partial(gv.Digraph, format='svg')

In [186]:
style = {
    'graph': {
        'label': 'Tree of Life',
        'labelloc' : 't',
        'fontsize': '16',
        'fontcolor': 'black',
        'bgcolor': 'white',
        'rankdir': 'TB',
    },
    'nodes': {
        'fontname': 'Helvetica',
        'shape': 'hexagon',
        'fontcolor': 'black',
        'color': 'white',
        'style': 'filled',
        'fillcolor': 'green',
    },
    'edges': {
        'style': 'dashed',
        'color': 'black',
        'arrowhead': 'open',
        'fontname': 'Courier',
        'fontsize': '12',
        'fontcolor': 'white',
    }
}

In [187]:
#Here we'll build some helper functions
def add_nodes(graph, nodes):
    for n in nodes:
        if isinstance(n, tuple):
            graph.node(n[0], **n[1])
        else:
            graph.node(n)
    return graph

def add_edges(graph, edges):
    for e in edges:
        if isinstance(e[0], tuple):
            graph.edge(*e[0], **e[1])
        else:
            graph.edge(*e)
    return graph

def apply_style(graph, style):
    graph.graph_attr.update(
        ('graph' in style and style['graph']) or {}
    )
    graph.node_attr.update(
        ('nodes' in style and style['nodes']) or {}
    )
    graph.edge_attr.update(
        ('edges' in style and style['edges']) or {}
    )
    return graph


In [188]:
#Now let's create a tuple for each clade which lists its name, parent clade, and rough age or origination
precambrian = ('Pre-cambrian life', None, 600) # 550-620 http://sci.waikato.ac.nz/evolution/AnimalEvolution.shtml
insects = ('Insects', precambrian[0], 480) # Wiki: Evolution of insects
vertebrates = ('Vertebrates', precambrian[0], 525) # Wiki: Vertebrate
sharks = ('Sharks', vertebrates[0], 450) # 425-455 http://www.elasmo-research.org/education/evolution/geologic_time.htm 
tetrapods = ('Tetrapods', vertebrates[0], 390) #Wiki: Evolution of tetrapods
amniotes = ('Amniotes', tetrapods[0], 340) #Wiki: Tetrapod
amphibians = ('Amphibians', tetrapods[0], 370) #Wiki: Amphibian
sauropsids = ('Sauropsids', amniotes[0], 320) #Wiki: Sauropsida
synapsids = ('Synapsids', amniotes[0], 320) #Wiki: Evolution of mammals
archosaurs = ('Archosaurs', sauropsids[0], 250) #Wiki: Archosaur says late Permian or early Triassic
mammals = ('Mammals', synapsids[0], 180) #Estimate from tree of life
therian = ('Therian mammals', mammals[0], 170) #Used Juramaia date (Wiki: Juramaia) + 10 million years (genetic analysis suggets older)
monotremes = ('Monotremes', mammals[0], 64) #Estimate from tree of life
placenta = ('Placenta mammals', therian[0], 160) #Estimate from tree of life
marsupials = ('Marsupials', therian[0], 160) #Estimate from tree of life
echidnas = ('Echidnas', monotremes[0], 30) #Wiki: Genetic evidence has determined that echidnas diverged from the platypus lineage as recently as 19-48M,
#when they made their transition from semi-aquatic to terrestrial lifestyle.


In [189]:
#Make a list of all the nodes
all_nodes = {precambrian, vertebrates, insects, sharks, tetrapods, amniotes, amphibians, sauropsids, archosaurs, mammals,
             synapsids, therian, monotremes, placenta, marsupials, echidnas}

# Now let's make a nested dictionary for our Tree of Life (tol)
tol = {node[0] : {'Age': node[2], 'Parent' : node[1]} for node in all_nodes}

In [190]:
def node_manager(date, nodes=all_nodes, parent=precambrian):
    '''This function allows you to find all the nodes (and their respective edges) given a particular date.
    
    Date is in millions of years ago.'''
    #Find the date of the common ancestor for this clade
    keeper_nodes=[]
    edges = []
    #for x in range(len(nodes)):
        
    for node in nodes:
        if node[2] > date and node[2] < parent[2]: #if the node's date is older than the date selected, but
            #younger than the parent node, add it to the keeper list
            keeper_nodes.append(node[0])
            try:
                edges.append(tuple([node[1], node[0]]))
            except TypeError:
                print("Edge {} is giving you a NoneType".format(node))    
    return (keeper_nodes, edges)

In [191]:
nodes, edges = node_manager(50)

In [192]:
treeoflife = add_edges(
    add_nodes(digraph(), nodes),
    edges
)

In [193]:
treeoflife = apply_style(treeoflife, style)
treeoflife.render('img/tol')

'img/tol.svg'